In [1]:
from room import Room
from person import Person
from action import Action
import metacode_helpers

In [2]:
import pandas as pd
import numpy as np

In [3]:
# initialise all actions
ALL_POSSIBLE_ACTIONS = metacode_helpers.initialise_all_actions()
ALL_CONVERSATION_ACTIONS = [action for action in ALL_POSSIBLE_ACTIONS if action.get_action_type() == "conversation"]
ALL_ROOM_ACTIONS = [action for action in ALL_POSSIBLE_ACTIONS if action.get_action_type() == "room"] 

In [4]:
# Create some rooms
HALL = Room("Hall")
KITCHEN = Room("Kitchen")
LOUNGE = Room("Lounge")

# Add adjacencies to the rooms (HALL is adjacent to KITCHEN and LOUNGE)
HALL.add_adjacent_room(KITCHEN)
HALL.add_adjacent_room(LOUNGE)

# Assign the Hall to be the entry to the house
HOUSE_ENTRY_ROOM = HALL 

ROOMS = [HALL, KITCHEN, LOUNGE]

In [5]:
names = metacode_helpers.get_names(num_names=32)
np.random.seed(42)
ALL_PEOPLE = []
for personality_number in range(32):
    personality_vector = np.random.uniform(low = -1, high = 1, size = 5)
    initial_emotional_state_vector = np.random.uniform(low = -1, high = 1, size = 3)
    initial_location_state = np.random.choice(ROOMS)
    new_person = Person(name = names[personality_number], location_state=initial_location_state, all_possible_actions=ALL_POSSIBLE_ACTIONS, personality_vector=personality_vector, emotional_state_vector=initial_emotional_state_vector, conversation_partner=None)
    ALL_PEOPLE.append(new_person)

In [6]:
ALL_PEOPLE

[Robin,
 Patrick,
 Ronnie,
 Jacquelyn,
 Phyllis,
 Agnes,
 Dorothy,
 Shawn,
 David,
 Donald,
 David,
 Michael,
 Robert,
 Donald,
 Maria,
 Edith,
 Nola,
 Jason,
 David,
 Martin,
 Andrew,
 Johnny,
 Elisabeth,
 Catherine,
 Vicki,
 Laquita,
 Jerry,
 John,
 Brian,
 James,
 Mauricio,
 Lillian]

In [7]:
ALL_POSSIBLE_ACTIONS

array([shouts, shares, leaves_conversation, makes_smalltalk,
       makes_deeptalk, gives_controversial_opinion, changes_the_topic,
       shows_interest, shows_concern, laughs, jokes, gives_an_insult,
       brags, yawns, supports, rants, stands_in_corner, leaves_room,
       starts_conversation], dtype=object)

In [8]:
ALL_ROOM_ACTIONS

[stands_in_corner, leaves_room, starts_conversation]

In [9]:
from collections import defaultdict

action_log = defaultdict(list)
person_before_action_log = defaultdict(list)
person_after_action_log = defaultdict(list)
partner_before_action_log = defaultdict(list)
partner_after_action_log = defaultdict(list)
room_log = defaultdict(list)

def log_snapshot(log, snapshot, turn_number, round_number):
    log["turn_number"].append(turn_number)
    log["round_number"].append(round_number)
    for key, val in snapshot.items():
        log[key].append(val)

turn_number = 0
for round_number in range(10):
    for person in ALL_PEOPLE:
        partner = person.get_conversation_partner()
        log_snapshot(person_before_action_log, person.get_snapshot(), turn_number, round_number)
        if person.has_conversation_partner():
            log_snapshot(partner_before_action_log, person.get_conversation_partner().get_snapshot(), turn_number, round_number)
            #print(partner.get_snapshot())

        # The person can start a conversation only if there's someone free to chat in their current room.
        available_room_act = ALL_ROOM_ACTIONS if person.get_location_state().is_someone_free_to_chat(person) else [action for action in ALL_ROOM_ACTIONS if action.get_name() != "starts_conversation"]

        # If in a conversation, the person can select any action, including leaving.
        action = person.action_selection(available_conv_act=ALL_CONVERSATION_ACTIONS) if person.has_conversation_partner() else person.action_selection(available_room_act=available_room_act)
        if partner is not None:
            print(person.get_name(), partner.get_name(), action.get_name(),person.get_location_state().get_name())
        else:
            print(person.get_name(), action.get_name(),person.get_location_state().get_name())
        # If in a conversation, update the person & conversation partner's emotional state according to the action
        if person.has_conversation_partner():
            # Change this person's emotional state 
            person.update_emotional_state_vector(action, isOwnAction = True)
            # Change the partner's emotional state 
            partner.update_emotional_state_vector(action, isOwnAction = False)

            if action.get_name() == "leaves_conversation":
                # The conversation has ended, so both people now don't have conversation partners.
                person.set_conversation_partner(None)
                partner.set_conversation_partner(None)
                
        else: # Otherwise, deal with the room action
            if action.get_name() == "leaves_room":
                next_room = person.get_next_room()
                person.move_to_room(next_room)          
            if action.get_name() == "starts_conversation":
                # NOTE whether this is possible is handled by the action selection function
                # Find a person in the room who is free to chat
                current_room = person.get_location_state()
                partner = current_room.get_new_conversation_partner_for(person)
                person.set_conversation_partner(partner)
                partner.set_conversation_partner(person)
        
        log_snapshot(action_log, action.get_snapshot(), turn_number, round_number)    
        log_snapshot(person_after_action_log, person.get_snapshot(), turn_number, round_number)
        if partner:
            log_snapshot(partner_after_action_log, partner.get_snapshot(), turn_number, round_number)

        message = person.get_name(), ' '.join(action.get_name().split("_"))
        print(round_number)
        print(turn_number)
        print(message)
        turn_number += 1

Robin starts_conversation Lounge
{Ronnie, Jason, Martin, Jerry, Brian, Laquita, Elisabeth, Vicki, Robin, Dorothy}
0
Patrick stands_in_corner Hall
1
Ronnie leaves_room Lounge
2
Jacquelyn leaves_room Kitchen
3
Phyllis starts_conversation Kitchen
{Andrew, Phyllis, Nola, Edith, James, Mauricio, Lillian, Donald, Agnes, Catherine}
4
Agnes leaves_room Kitchen
5
Dorothy leaves_room Lounge
6
Shawn leaves_room Hall
7
David starts_conversation Hall
{Ronnie, Maria, John, David, David, Michael, Jacquelyn, Donald, Robert, Johnny, Agnes, Patrick, David, Dorothy}
8
Donald stands_in_corner Kitchen
9
David starts_conversation Hall
{Ronnie, Maria, John, David, David, Michael, Jacquelyn, Donald, Robert, Johnny, Agnes, Patrick, David, Dorothy}
10
Michael leaves_room Hall
11
Robert David shows_concern Hall
12
Donald starts_conversation Hall
{Ronnie, Maria, John, David, David, Jacquelyn, Donald, Robert, Johnny, Agnes, Patrick, David, Dorothy}
13
Maria starts_conversation Hall
{Ronnie, Maria, John, David, Dav

Mauricio Lillian laughs Kitchen
190
Lillian Mauricio laughs Kitchen
191
Robin Elisabeth supports Lounge
192
Patrick Brian shouts Hall
193
Ronnie David makes_deeptalk Hall
194
Jacquelyn Donald rants Hall
195
Phyllis Edith yawns Kitchen
196
Agnes Maria shows_concern Hall
197
Dorothy Martin laughs Lounge
198
Shawn Catherine makes_deeptalk Kitchen
199
David Ronnie makes_smalltalk Hall
200
Donald David shows_interest Hall
201
David Donald gives_controversial_opinion Hall
202
Michael Laquita laughs Lounge
203
Robert Nola shouts Kitchen
204
Donald Jacquelyn rants Hall
205
Maria Agnes rants Hall
206
Edith Phyllis gives_an_insult Kitchen
207
Nola Robert shows_concern Kitchen
208
Jason David makes_deeptalk Hall
209
David Jason brags Hall
210
Martin Dorothy gives_an_insult Lounge
211
Andrew James leaves_conversation Kitchen
212
Johnny Jerry supports Lounge
213
Elisabeth Robin brags Lounge
214
Catherine Shawn supports Kitchen
215
Vicki John brags Hall
216
Laquita Michael jokes Lounge
217
Jerry Joh

In [12]:
pd.DataFrame(action_log)

,turn_number,round_number,name
0,0,0,shouts
1,1,0,shouts
2,2,0,brags
3,3,0,gives_controversial_opinion
4,4,0,shares
...,...,...,...
95,95,9,rants
96,96,9,jokes
97,97,9,changes_the_topic
98,98,9,laughs
